In [1]:
%load_ext autoreload
%autoreload 2

In [33]:
from argparse import ArgumentParser, Namespace
from random import choices
import pytorch_lightning as pl
from typing import Callable, List, Optional, Sequence, Union
import squidpy as sq
import torch
from torch_geometric.loader import RandomNodeSampler
import pandas as pd
from torch_geometric.data import Data
from anndata import AnnData
from utils import adata2data
from data.node_datamodule import NodeDataModule
from models.graph_embedding import GraphEmbedding
from models.non_linear_ncem import NonLinearNCEM
from models.linear_ncem import LinearNCEM

In [34]:
#Mibitof
# adata = sq.datasets.mibitof()
# #feature_name=adata.obs.keys()[0] #Use for IMC dataset
# feature_name='Cluster'
# num_features=len(set(adata.obs[feature_name]))
# encoder_hidden_dims=30
# decoder_hidden_dims=30
# num_vars=adata.X.shape[1]

#IMC
adata = sq.datasets.imc()
feature_name=adata.obs.keys()[0] #Use for IMC dataset
num_features=len(set(adata.obs[feature_name]))
num_vars=adata.X.shape[1]


In [43]:
dm = NodeDataModule(adata=adata,feature_name=feature_name,adata2data_fn=adata2data, num_workers = 8, batch_size=100)

In [38]:
dm.setup()

In [39]:
for batch in dm.train_dataloader():
    print(batch)

Data(num_nodes=42, edge_index=[2, 7], x=[42, 11], y=[42, 34])
Data(num_nodes=43, edge_index=[2, 0], x=[43, 11], y=[43, 34])
Data(num_nodes=43, edge_index=[2, 0], x=[43, 11], y=[43, 34])
Data(num_nodes=50, edge_index=[2, 3], x=[50, 11], y=[50, 34])
Data(num_nodes=51, edge_index=[2, 0], x=[51, 11], y=[51, 34])
Data(num_nodes=53, edge_index=[2, 4], x=[53, 11], y=[53, 34])
Data(num_nodes=54, edge_index=[2, 3], x=[54, 11], y=[54, 34])
Data(num_nodes=53, edge_index=[2, 4], x=[53, 11], y=[53, 34])
Data(num_nodes=40, edge_index=[2, 2], x=[40, 11], y=[40, 34])
Data(num_nodes=48, edge_index=[2, 0], x=[48, 11], y=[48, 34])
Data(num_nodes=41, edge_index=[2, 0], x=[41, 11], y=[41, 34])
Data(num_nodes=38, edge_index=[2, 4], x=[38, 11], y=[38, 34])
Data(num_nodes=45, edge_index=[2, 6], x=[45, 11], y=[45, 34])
Data(num_nodes=60, edge_index=[2, 2], x=[60, 11], y=[60, 34])
Data(num_nodes=58, edge_index=[2, 2], x=[58, 11], y=[58, 34])
Data(num_nodes=58, edge_index=[2, 3], x=[58, 11], y=[58, 34])
Data(num

In [44]:
model = LinearNCEM(in_channels=num_features,out_channels=num_vars, lr=0.0001,weight_decay=0.000001)

In [45]:
gpu=False
if gpu:
    trainer:pl.Trainer = pl.Trainer(accelerator='gpu',max_epochs=100,log_every_n_steps=1)
else:
    trainer:pl.Trainer = pl.Trainer(accelerator='cpu',max_epochs=100,log_every_n_steps=1)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [46]:
trainer.fit(model,datamodule=dm)


  | Name        | Type            | Params
------------------------------------------------
0 | model_sigma | LinearSpatial   | 408   
1 | model_mu    | LinearSpatial   | 408   
2 | loss_module | GaussianNLLLoss | 0     
------------------------------------------------
816       Trainable params
0         Non-trainable params
816       Total params
0.003     Total estimated model params size (MB)


Epoch 99: 100%|██████████| 200/200 [00:08<00:00, 22.45it/s, loss=46.4, v_num=12, val_r2_score=-.378, val_loss=47.80]
